In [36]:
from elftools.elf.elffile import ELFFile
import frida
import sys
from time import sleep

In [37]:
def generate_function_list(binary):
    """
    Generate a list of all the functions defined by the target executable.
    """
    functions = []
    with open(binary, "rb") as f:
        elf = ELFFile(f)
        for section in elf.iter_sections():
            if section.header.sh_type == 'SHT_SYMTAB':
                for symbol in section.iter_symbols():
                    if symbol.entry.st_info.type == 'STT_FUNC':
                        functions.append(symbol.name)
    return functions

In [38]:
binary_path = "./test/test"
args = ["arg1", "arg2"]

function_list = generate_function_list(binary_path)
print(function_list)

['deregister_tm_clones', 'register_tm_clones', '__do_global_dtors_aux', 'frame_dummy', '__libc_start_main@GLIBC_2.34', 'puts@GLIBC_2.2.5', 'h', '_fini', 'f', '_start', 'g', 'main', '__cxa_finalize@GLIBC_2.2.5', '_init']


In [42]:
entries = []

def on_message(message, data):
    print(message)
    if message["type"] == "send" and message["payload"] != "done":
        function_name = message["payload"]["function"]
        function_args = message["payload"]["args"]
        entries.append((function_name, function_args))

# Run the binary
process = frida.spawn(binary_path, argv=[binary_path] + args)

sleep(5)

session = frida.attach(process)
for f in ['f','g','h']:
    script = session.create_script("""
        console.log(Module.findExportByName(null, '"""+f+"""'))
        Interceptor.attach(Module.findExportByName(null, '"""+f+"""'), {
            onEnter(args) {
                send({function: """+f+""", args: args})
            },
            onLeave(retval) {
                send({function: """+f+""", ret: retval})

            }
        });
    """)
    
    script.on("message", on_message)
    script.load()

frida.resume(process)

null
{'type': 'error', 'description': 'Error: expected a pointer', 'stack': 'Error: expected a pointer\n    at value (frida/runtime/core.js:367)\n    at <eval> (/script1.js:11)', 'fileName': 'frida/runtime/core.js', 'lineNumber': 367, 'columnNumber': 1}
null
{'type': 'error', 'description': 'Error: expected a pointer', 'stack': 'Error: expected a pointer\n    at value (frida/runtime/core.js:367)\n    at <eval> (/script2.js:11)', 'fileName': 'frida/runtime/core.js', 'lineNumber': 367, 'columnNumber': 1}
null
{'type': 'error', 'description': 'Error: expected a pointer', 'stack': 'Error: expected a pointer\n    at value (frida/runtime/core.js:367)\n    at <eval> (/script3.js:11)', 'fileName': 'frida/runtime/core.js', 'lineNumber': 367, 'columnNumber': 1}
Hello
